In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from numpy import mean
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
import torch
import torch.nn as nn
from torch.utils.data import  TensorDataset, DataLoader
from imblearn.over_sampling import SMOTE

df = pd.read_csv('final.csv')
df = pd.get_dummies(df, columns=['burden', 'cardio', 'diab', 'male', 'educ','ss_type'])
y = df['target_es']
X = df.drop(['target_es'], axis=1, inplace=False)
X_train, X_test_pre, es, y_test = train_test_split(X, y, test_size = 0.3, random_state = 2, stratify = y)
ap=X_train['target_ap']
X_train_pre=X_train.drop(['target_ap'],axis=1, inplace=False)


X_test=X_test_pre.drop(['target_ap'], axis=1, inplace=False)
X_train_pre_tr,X_train_pre_te,ap_tr,ap_te=train_test_split(X_train_pre,ap, test_size=0.3, random_state=2, stratify=ap)



new_X_train_pre_tr, new_ap_tr = SMOTE(random_state = 1).fit_resample(X_train_pre_tr, ap_tr)
ap_rf = RandomForestClassifier(criterion='gini',max_depth=6, random_state=42)
ap_nb = GaussianNB()
ap_gb = GradientBoostingClassifier(random_state=42)
ap_ada = AdaBoostClassifier(n_estimators=100)

ap_rf.fit(new_X_train_pre_tr,new_ap_tr)
ap_nb.fit(new_X_train_pre_tr,new_ap_tr)
ap_gb.fit(new_X_train_pre_tr,new_ap_tr)
ap_ada.fit(new_X_train_pre_tr,new_ap_tr)

pred_ap_rf=ap_rf.predict(X_train_pre_te)
pred_ap_nb=ap_nb.predict(X_train_pre_te)
pred_ap_gb=ap_gb.predict(X_train_pre_te)
pred_ap_ada=ap_ada.predict(X_train_pre_te)

print(classification_report(ap_te, pred_ap_rf))
print(classification_report(ap_te, pred_ap_nb))
print(classification_report(ap_te, pred_ap_gb))
print(classification_report(ap_te, pred_ap_ada))

pred = np.array([pred_ap_rf, pred_ap_nb, pred_ap_gb, pred_ap_ada])
pred = np.transpose(pred)

lr_ap = XGBClassifier(seed = 0, n_jobs = -1, learning_rate = 0.1, n_estimators = 100, max_depth = 3)

lr_ap.fit(pred, ap_te)

final_ap = lr_ap.predict(pred)
print('최종 ap 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(ap_te, final_ap)))
print(classification_report(ap_te, final_ap))

new_X_train, new_es = SMOTE(random_state = 1).fit_resample(X_train, es)

es_rf = RandomForestClassifier(criterion='gini',max_depth=6, random_state=42)
es_nb=GaussianNB()
es_gb = GradientBoostingClassifier(random_state=42)
es_ada = AdaBoostClassifier(n_estimators=100)

es_rf.fit(new_X_train,new_es)
es_nb.fit(new_X_train,new_es)
es_gb.fit(new_X_train,new_es)
es_ada.fit(new_X_train,new_es)


pred_apes_rf=ap_rf.predict(X_test)
pred_apes_nb=ap_nb.predict(X_test)
pred_apes_gb=ap_gb.predict(X_test)
pred_apes_ada=ap_ada.predict(X_test)
pred2 = np.array([pred_apes_rf, pred_apes_nb, pred_apes_gb, pred_apes_ada])
pred2 = np.transpose(pred2)

predicted_ap_pre = lr_ap.predict(pred2)

colnames = ["target_ap"]
rowindex=X_test.index
predicted_ap = pd.DataFrame(predicted_ap_pre,index=rowindex,columns=colnames)
X_test_comb=pd.concat([X_test,predicted_ap],axis=1)


pred_es_rf=es_rf.predict(X_test_comb)
pred_es_nb=es_nb.predict(X_test_comb)
pred_es_gb=es_gb.predict(X_test_comb)
pred_es_ada=es_ada.predict(X_test_comb)

print(classification_report(y_test, pred_es_rf))
print(classification_report(y_test, pred_es_nb))
print(classification_report(y_test, pred_es_gb))
print(classification_report(y_test, pred_es_ada))

pred3 = np.array([pred_es_rf, pred_es_nb, pred_es_gb, pred_es_ada])
pred3 = np.transpose(pred3)

lr_final = XGBClassifier(seed = 0, n_jobs = -1, learning_rate = 0.1, n_estimators = 100, max_depth = 3)
lr_final.fit(pred3, y_test)
final_es = lr_final.predict(pred3)
print('최종 es 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, final_es)))
print(classification_report(y_test, final_es))